# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 5 2022 12:19PM,241094,12,SL-Bonne,Doctors Scientific Organica,Released
1,Jul 5 2022 12:00PM,241092,18,EMERSA-8234698,"Emersa Waterbox, LLC",Released
2,Jul 5 2022 12:00PM,241093,12,U0957,"Uniderm USA, Inc",Released
3,Jul 5 2022 11:42AM,241087,20,8183904,"ACI Healthcare USA, Inc.",Released
4,Jul 5 2022 11:42AM,241087,20,8198423-BO,"ACI Healthcare USA, Inc.",Released
5,Jul 5 2022 11:42AM,241087,20,8198419-BO,"ACI Healthcare USA, Inc.",Released
6,Jul 5 2022 11:42AM,241087,20,8199592-BO,"ACI Healthcare USA, Inc.",Released
7,Jul 5 2022 11:42AM,241087,20,8213039,"ACI Healthcare USA, Inc.",Released
8,Jul 5 2022 11:42AM,241087,20,8213048,"ACI Healthcare USA, Inc.",Released
9,Jul 5 2022 11:42AM,241087,20,8213054,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
64,241088,Released,4
65,241089,Released,1
66,241092,Released,1
67,241093,Released,1
68,241094,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241088,NaN,4.0
241089,NaN,1.0
241092,NaN,1.0
241093,NaN,1.0
241094,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241010,0.0,60.0
241016,0.0,60.0
241017,0.0,60.0
241019,0.0,55.0
241022,60.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241010,0,60
241016,0,60
241017,0,60
241019,0,55
241022,60,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241010,0,60
1,241016,0,60
2,241017,0,60
3,241019,0,55
4,241022,60,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241010,,60
1,241016,,60
2,241017,,60
3,241019,,55
4,241022,60,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 5 2022 12:19PM,241094,12,Doctors Scientific Organica
1,Jul 5 2022 12:00PM,241092,18,"Emersa Waterbox, LLC"
2,Jul 5 2022 12:00PM,241093,12,"Uniderm USA, Inc"
3,Jul 5 2022 11:42AM,241087,20,"ACI Healthcare USA, Inc."
33,Jul 5 2022 11:38AM,241089,19,"Emersa Waterbox, LLC"
34,Jul 5 2022 11:37AM,241088,10,"Methapharm, Inc."
38,Jul 5 2022 11:31AM,241086,10,"ClearSpec, LLC"
39,Jul 5 2022 11:14AM,241085,10,"ClearSpec, LLC"
40,Jul 5 2022 11:00AM,241083,20,"ACI Healthcare USA, Inc."
63,Jul 5 2022 10:56AM,241082,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 5 2022 12:19PM,241094,12,Doctors Scientific Organica,,1
1,Jul 5 2022 12:00PM,241092,18,"Emersa Waterbox, LLC",,1
2,Jul 5 2022 12:00PM,241093,12,"Uniderm USA, Inc",,1
3,Jul 5 2022 11:42AM,241087,20,"ACI Healthcare USA, Inc.",,30
4,Jul 5 2022 11:38AM,241089,19,"Emersa Waterbox, LLC",,1
5,Jul 5 2022 11:37AM,241088,10,"Methapharm, Inc.",,4
6,Jul 5 2022 11:31AM,241086,10,"ClearSpec, LLC",,1
7,Jul 5 2022 11:14AM,241085,10,"ClearSpec, LLC",,1
8,Jul 5 2022 11:00AM,241083,20,"ACI Healthcare USA, Inc.",,23
9,Jul 5 2022 10:56AM,241082,10,ISDIN Corporation,,63


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 12:19PM,241094,12,Doctors Scientific Organica,1,
1,Jul 5 2022 12:00PM,241092,18,"Emersa Waterbox, LLC",1,
2,Jul 5 2022 12:00PM,241093,12,"Uniderm USA, Inc",1,
3,Jul 5 2022 11:42AM,241087,20,"ACI Healthcare USA, Inc.",30,
4,Jul 5 2022 11:38AM,241089,19,"Emersa Waterbox, LLC",1,
5,Jul 5 2022 11:37AM,241088,10,"Methapharm, Inc.",4,
6,Jul 5 2022 11:31AM,241086,10,"ClearSpec, LLC",1,
7,Jul 5 2022 11:14AM,241085,10,"ClearSpec, LLC",1,
8,Jul 5 2022 11:00AM,241083,20,"ACI Healthcare USA, Inc.",23,
9,Jul 5 2022 10:56AM,241082,10,ISDIN Corporation,63,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 12:19PM,241094,12,Doctors Scientific Organica,1,
1,Jul 5 2022 12:00PM,241092,18,"Emersa Waterbox, LLC",1,
2,Jul 5 2022 12:00PM,241093,12,"Uniderm USA, Inc",1,
3,Jul 5 2022 11:42AM,241087,20,"ACI Healthcare USA, Inc.",30,
4,Jul 5 2022 11:38AM,241089,19,"Emersa Waterbox, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 12:19PM,241094,12,Doctors Scientific Organica,1.0,NaN
1,Jul 5 2022 12:00PM,241092,18,"Emersa Waterbox, LLC",1.0,NaN
2,Jul 5 2022 12:00PM,241093,12,"Uniderm USA, Inc",1.0,NaN
3,Jul 5 2022 11:42AM,241087,20,"ACI Healthcare USA, Inc.",30.0,NaN
4,Jul 5 2022 11:38AM,241089,19,"Emersa Waterbox, LLC",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 12:19PM,241094,12,Doctors Scientific Organica,1.0,0.0
1,Jul 5 2022 12:00PM,241092,18,"Emersa Waterbox, LLC",1.0,0.0
2,Jul 5 2022 12:00PM,241093,12,"Uniderm USA, Inc",1.0,0.0
3,Jul 5 2022 11:42AM,241087,20,"ACI Healthcare USA, Inc.",30.0,0.0
4,Jul 5 2022 11:38AM,241089,19,"Emersa Waterbox, LLC",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,10847294,1421.0,68.0
12,1205327,56.0,28.0
15,1446390,60.0,43.0
18,241092,1.0,0.0
19,482146,2.0,0.0
20,723232,63.0,19.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,10847294,1421.0,68.0
1,12,1205327,56.0,28.0
2,15,1446390,60.0,43.0
3,18,241092,1.0,0.0
4,19,482146,2.0,0.0
5,20,723232,63.0,19.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,1421.0,68.0
1,12,56.0,28.0
2,15,60.0,43.0
3,18,1.0,0.0
4,19,2.0,0.0
5,20,63.0,19.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1421.0
1,12,Released,56.0
2,15,Released,60.0
3,18,Released,1.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20
Status,,,,,,
Executing,68.0,28.0,43.0,0.0,0.0,19.0
Released,1421.0,56.0,60.0,1.0,2.0,63.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20
0,Executing,68.0,28.0,43.0,0.0,0.0,19.0
1,Released,1421.0,56.0,60.0,1.0,2.0,63.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20
0,Executing,68.0,28.0,43.0,0.0,0.0,19.0
1,Released,1421.0,56.0,60.0,1.0,2.0,63.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()